In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from google.colab import files

In [ ]:
# 1 Veri setini yükleme
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])

Saving Steel_industry_data.csv to Steel_industry_data.csv


In [ ]:
df.head(12)

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load
5,01/01/2018 01:30,3.28,3.56,0.0,0.0,67.76,100.0,5400,Weekday,Monday,Light_Load
6,01/01/2018 01:45,3.60,4.14,0.0,0.0,65.62,100.0,6300,Weekday,Monday,Light_Load
7,01/01/2018 02:00,3.60,4.28,0.0,0.0,64.37,100.0,7200,Weekday,Monday,Light_Load
8,01/01/2018 02:15,3.28,3.64,0.0,0.0,66.94,100.0,8100,Weekday,Monday,Light_Load
9,01/01/2018 02:30,3.78,4.72,0.0,0.0,62.51,100.0,9000,Weekday,Monday,Light_Load


In [ ]:
# 2 Tarih-saat sütununu datetime formatına çevirme
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')
df = df.sort_values('date')


In [ ]:
df.head(12)

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
95,2018-01-01 00:00:00,3.42,3.46,0.0,0.0,70.30,100.0,0,Weekday,Monday,Light_Load
0,2018-01-01 00:15:00,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,2018-01-01 00:30:00,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,2018-01-01 00:45:00,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,2018-01-01 01:00:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,2018-01-01 01:15:00,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load
5,2018-01-01 01:30:00,3.28,3.56,0.0,0.0,67.76,100.0,5400,Weekday,Monday,Light_Load
6,2018-01-01 01:45:00,3.60,4.14,0.0,0.0,65.62,100.0,6300,Weekday,Monday,Light_Load
7,2018-01-01 02:00:00,3.60,4.28,0.0,0.0,64.37,100.0,7200,Weekday,Monday,Light_Load
8,2018-01-01 02:15:00,3.28,3.64,0.0,0.0,66.94,100.0,8100,Weekday,Monday,Light_Load


In [ ]:
# 3 Gerekli sütunları seçme ve ek özellikler çıkarma
df['hour'] = df['date'].dt.hour
df['day'] = df['date'].dt.dayofweek
features = ['Usage_kWh', 'hour', 'day']
data = df[features]


In [ ]:
# Adım 4: Ayrı bir scaler ile sadece Usage_kWh için normalizasyon
usage_scaler = MinMaxScaler()
usage_scaled = usage_scaler.fit_transform(data[['Usage_kWh']])



In [ ]:
data.head(36)

,Usage_kWh,hour,day
95,3.42,0,0
0,3.17,0,0
1,4.00,0,0
2,3.24,0,0
3,3.31,1,0
4,3.82,1,0
5,3.28,1,0
6,3.60,1,0
7,3.60,2,0
8,3.28,2,0


In [ ]:
# Adım 5: Tüm veriyi ölçeklendirme (normalizasyon)
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

In [ ]:
# Adım 6: Zaman serisi penceresi oluşturma fonksiyonu
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

In [ ]:

# Adım 7: Pencereleme ve veri hazırlığı
sequence_length = 96
X, y = create_sequences(scaled_data, sequence_length)

In [ ]:
# Adım 8: Eğitim ve test setine ayırma
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]



In [ ]:

# Adım 9: LSTM modelini oluşturma (Dropout ile)
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, len(features)), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Adım 10: Early Stopping ve ReduceLROnPlateau callback'lerini tanımlama
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.0001)

In [ ]:
# Adım 11: Modeli eğitme
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1,
                    callbacks=[early_stopping, reduce_lr], verbose=1)

Epoch 1/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 23s 22ms/step - loss: 0.0281 - val_loss: 0.0081 - learning_rate: 0.0010
Epoch 2/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - loss: 0.0120 - val_loss: 0.0064 - learning_rate: 0.0010
Epoch 3/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - loss: 0.0094 - val_loss: 0.0057 - learning_rate: 0.0010
Epoch 4/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - loss: 0.0086 - val_loss: 0.0049 - learning_rate: 0.0010
Epoch 5/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - loss: 0.0079 - val_loss: 0.0046 - learning_rate: 0.0010
Epoch 6/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - loss: 0.0077 - val_loss: 0.0044 - learning_rate: 0.0010
Epoch 7/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - loss: 0.0074 - val_loss: 0.0045 - learning_rate: 0.0010
Epoch 8/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - loss: 0.0073 - val_loss: 0.0045 - learning_rate: 0.0010
Epoch 9/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - loss: 0.0068 - val_loss: 0.0043 - learning_rate:

In [ ]:
# Adım 12: Tahmin yapma
y_pred = model.predict(X_test)


219/219 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step


In [ ]:
# Adım 13: Normalizasyonu geri alma (sadece Usage_kWh için)
y_test_original = usage_scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
y_pred_original = usage_scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()

In [ ]:
# Adım 14: Model değerlendirme (MSE ve R² skoru)
mse = mean_squared_error(y_test_original, y_pred_original)
r2 = r2_score(y_test_original, y_pred_original)
print(f"Mean Squared Error: {mse}")
print(f"R² Skoru: {r2:.4f}")

Mean Squared Error: 85.35888404128801
R² Skoru: 0.9134


In [ ]:
# Adım 15: Saatlik tahminler için veriyi gruplama
hourly_df = df.groupby([df['date'].dt.date, 'hour'])['Usage_kWh'].mean().reset_index()
hourly_actual = hourly_df.tail(24)['Usage_kWh'].values


In [ ]:
# Adım 16: 24 saatlik tahmin
last_sequence = scaled_data[-sequence_length:].reshape(1, sequence_length, len(features))
next_96_pred = []
for _ in range(96):
    pred = model.predict(last_sequence)[0, 0]
    next_96_pred.append(pred)
    new_row = last_sequence[0, -1].copy()
    new_row[0] = pred
    new_row[1] = (new_row[1] + 0.25) % 24
    last_sequence = np.roll(last_sequence, -1, axis=1)
    last_sequence[0, -1] = new_row

In [ ]:
# Adım 17: 24 saatlik tahmin için normalizasyonu geri alma
next_96_pred_original = usage_scaler.inverse_transform(np.array(next_96_pred).reshape(-1, 1)).flatten()

In [ ]:
# Adım 18: Saatlik ortalamaları hesaplama ve sonuçları yazdırma
hourly_pred = [next_96_pred_original[i:i+4].mean() for i in range(0, 96, 4)]
print(f"24 Saatlik Toplam Tahmin: {sum(hourly_pred):.2f} kWh")
print("Saatlik Tahminler:")
for hour, pred in enumerate(hourly_pred):
    print(f"Saat {hour:02d}:00 - {pred:.2f} kWh")

In [ ]:
# Adım 19: Görselleştirme - 15 dakikalık tahmin grafiği
plt.figure(figsize=(12, 6))
plt.plot(y_test_original[:96], label='Gerçek Değerler (15 dk)')
plt.plot(y_pred_original[:96], label='Tahminler (15 dk)')
plt.xlabel('Zaman (15 dakikalık aralıklar)')
plt.ylabel('Tüketim (kWh)')
plt.title('Gerçek vs Tahmin Edilen Tüketim (15 dakikalık)')
plt.legend()
plt.show()

In [ ]:
# Adım 20: Görselleştirme - Saatlik tahmin grafiği
plt.figure(figsize=(12, 6))
plt.plot(range(24), hourly_actual, label='Gerçek Saatlik Ortalama (Son 24 saat)')
plt.plot(range(24), hourly_pred, label='Tahmin Edilen Saatlik Ortalama')
plt.xlabel('Saat')
plt.ylabel('Tüketim (kWh)')
plt.title('Saatlik Gerçek vs Tahmin Edilen Tüketim')
plt.legend()
plt.show()

In [ ]:
# Adım 21: Görselleştirme - Eğitim sürecindeki kayıp grafiği
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kayıp (MSE)')
plt.title('Eğitim Sürecindeki Kayıp')
plt.legend()
plt.show()